In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from pmdarima.arima import auto_arima

import warnings
warnings.filterwarnings(action='ignore')


In [2]:
dataset = pd.read_csv('../../data/funda_train.csv')

In [3]:
dataset['year_month'] = dataset['transacted_date'].str.slice(stop=7)

In [4]:
""" drop_features = ['installment_term', 'region', 'type_of_business','transacted_date','card_company','transacted_time']
dataset = dataset.drop(drop_features, axis=1) """

In [4]:
subset = dataset[['store_id','amount','year_month','card_id']]
subset

refund = subset[subset['amount'] < 0]
refund

,store_id,amount,year_month,card_id
41,0,-8571.428571,2016-06,40
347,0,-1857.142857,2016-08,285
731,0,-2000.000000,2016-10,473
831,0,-85.714286,2016-11,230
944,0,-57.142857,2016-11,138
...,...,...,...,...
6556242,2136,-13428.571429,2019-02,4663626
6556448,2136,-6928.571429,2019-02,4663760
6556485,2136,-5571.428571,2019-02,4663779
6556489,2136,-8571.428571,2019-02,4663780


In [5]:
dataset.iloc[refund.index[0] -1 : refund.index[0] + 1]

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,year_month
40,0,40,a,2016-06-10,17:25,2,NaN,기타 미용업,8571.428571,2016-06
41,0,40,a,2016-06-10,17:26,2,NaN,기타 미용업,-8571.428571,2016-06


In [6]:
refund_case_1 = []
for i in tqdm(range(len(refund.index))):
    subset = dataset.iloc[refund.index[i]-1 : refund.index[i] + 1]
    if subset['amount'].sum() == 0:
        refund_case_1.append(refund.index[i]) # 환불 건의 index 를 추출합니다. index 41과 같은 건을 모두 추출
    else:
        pass

100%|██████████| 73100/73100 [00:27<00:00, 2634.60it/s]


In [7]:
refund_case_2 = []
for i in tqdm(range(len(refund.index))):
    subset = dataset.iloc[refund.index[i]-1 : refund.index[i] + 1]
    if subset['amount'].sum() == 0:
        refund_case_2.append(refund.index[i]-1)  # 환불 전 결제 건의 index 를 추출합니다. index 40과 같은 건을 모두 추출
    else:
        pass

100%|██████████| 73100/73100 [00:20<00:00, 3650.06it/s]


In [8]:
print("refund_case_1 : {refund} refund_case_2 : {payment}".format(refund = len(refund_case_1), payment = len(refund_case_2)))
if len(refund_case_1) == len(refund_case_2):
    print("두 케이스 동일")

refund_case_1 : 55347 refund_case_2 : 55347
두 케이스 동일


In [9]:
sub_refund = refund.drop(index = refund_case_1, axis = 0)

print("기존 환불 건수 : {before} 드랍 후 환불 건수 : {after}".format(before = len(refund), after = len(sub_refund)))

기존 환불 건수 : 73100 드랍 후 환불 건수 : 17753


In [10]:
sub_dataset = dataset.drop(index = refund_case_2, axis = 0)

In [11]:
refund_case_3 = []
for i in tqdm(range(len(sub_refund.index))):
    subset = sub_dataset.loc[(sub_dataset['amount'] == abs(sub_refund.iloc[i].amount)) & (sub_dataset['card_id'] == (sub_refund.iloc[i].card_id))] # 환불 전 결제 케이스
    refund_case_3.append(subset.index.min())

100%|██████████| 17753/17753 [11:22<00:00, 26.00it/s]


In [12]:
refund_case_3 = [x for x in refund_case_3 if np.isnan(x) == False] # 해당 조건을 만족하는 환불건만 있고 결제건이 없을때 Nan이 리스트에 들어가므로 난을 제거

- -- -------------

In [13]:
refund_case_4 = list(sub_refund.index)

In [14]:
print("refund_case_3 : {before} refund_case_4 : {after}".format(before = len(refund_case_3), after = len(refund_case_4))) # 환불 케이스

refund_case_3 : 17657 refund_case_4 : 17753


In [15]:
len(dataset) - 73100*2 # 우리가 원하는 결과

6410413

In [16]:
dataset = dataset.drop(index = refund_case_1, axis = 0)
len(dataset)

6501266

In [17]:
dataset = dataset.drop(index = refund_case_2, axis = 0)
len(dataset)

6445919

In [18]:
dataset = dataset.drop(index = refund_case_3, axis = 0)
len(dataset)

6428813

In [19]:
dataset = dataset.drop(index = refund_case_4, axis = 0)
len(dataset)

6411060

In [20]:
overlap_test = list(set(refund_case_1).intersection(refund_case_3)) # 중복 index가 있는지 확인
overlap_test

[]

In [21]:
print("해결하지 못한 환불 건수 : {case}".format(case = len(dataset) - 6410413))

해결하지 못한 환불 건수 : 647


In [22]:
subset_2 = dataset[['store_id','amount','year_month']]
subset_2

,store_id,amount,year_month
0,0,1857.142857,2016-06
1,0,857.142857,2016-06
2,0,2000.000000,2016-06
3,0,7857.142857,2016-06
4,0,2000.000000,2016-06
...,...,...,...
6556606,2136,3928.571429,2019-02
6556609,2136,4142.857143,2019-02
6556610,2136,4500.000000,2019-02
6556611,2136,571.428571,2019-02


In [60]:
se1=pd.DataFrame(data={'store_id': [],'amount' : [],'year_month' : []})

store = subset_2.loc[subset_2['store_id'] == 0]
se1['amount'] = store.groupby(['year_month'])['amount'].agg('sum').values
se1['year_month'] = store.groupby(['year_month'])['amount'].agg('sum').index
se1['store_id'] = 0


for i in tqdm(subset_2['store_id'].unique()[1:2]): # 시작은 1 고정

    tempset = pd.DataFrame(data={'store_id': [],'amount' : [],'year_month' : []})

    store_ = subset_2.loc[subset_2['store_id'] == i]

    tempset['amount'] = store_.groupby(['year_month'])['amount'].agg('sum').values
    tempset['year_month'] = store_.groupby(['year_month'])['amount'].agg('sum').index
    tempset['store_id'] = i
    print(tempset, i,"번째 템프셋")
    se1 = pd.concat([se1,tempset], axis = 0)

se1 = se1.reset_index(drop=True)

100%|██████████| 1/1 [00:00<00:00, 24.67it/s]

    store_id         amount year_month
0          1  137214.285714    2016-06
1          1  163000.000000    2016-07
2          1  118142.857143    2016-08
3          1   90428.571429    2016-09
4          1  118071.428571    2016-10
5          1  111857.142857    2016-11
6          1  115571.428571    2016-12
7          1  129642.857143    2017-01
8          1  160214.285714    2017-02
9          1  168428.571429    2017-03
10         1  152571.428571    2017-04
11         1  107500.000000    2017-05
12         1  110357.142857    2017-06
13         1  132571.428571    2017-07
14         1  107642.857143    2017-08
15         1  131357.142857    2017-09
16         1   80142.857143    2017-10
17         1  110142.857143    2017-11
18         1  100714.285714    2017-12
19         1  109571.428571    2018-01
20         1   94214.285714    2018-02
21         1  108357.142857    2018-03
22         1  108857.142857    2018-04
23         1   80500.000000    2018-05
24         1   78285.7142

In [23]:
finalset=pd.DataFrame(data={'store_id': [],'amount' : [],'year_month' : []})

store = subset_2.loc[subset_2['store_id'] == 0]
finalset['amount'] = store.groupby(['year_month'])['amount'].agg('sum').values
finalset['year_month'] = store.groupby(['year_month'])['amount'].agg('sum').index
finalset['store_id'] = 0


for i in tqdm(subset_2['store_id'].unique()[1:]): # 시작은 1 고정

    tempset = pd.DataFrame(data={'store_id': [],'amount' : [],'year_month' : []})

    store_ = subset_2.loc[subset_2['store_id'] == i]

    tempset['amount'] = store_.groupby(['year_month'])['amount'].agg('sum').values
    tempset['year_month'] = store_.groupby(['year_month'])['amount'].agg('sum').index
    tempset['store_id'] = i

    finalset = pd.concat([finalset,tempset], axis = 0)

finalset = finalset.reset_index(drop=True)

100%|██████████| 1966/1966 [00:41<00:00, 47.79it/s]


In [29]:
len(subset_2['store_id'].unique())

1967

In [24]:
finalset['datetime'] = pd.to_datetime(finalset['year_month'] + '-25', format='%Y-%m-%d')

In [25]:
finalset.to_csv('../../Data/finalset.csv')

In [9]:
finalset = pd.read_csv('../../Data/finalset.csv', index_col= 'Unnamed: 0')

In [11]:
finalset = finalset.drop('year_month', axis = 1)
finalset

,store_id,amount,datetime
0,0,7.470000e+05,2016-06-25
1,0,1.005000e+06,2016-07-25
2,0,8.715714e+05,2016-08-25
3,0,8.978571e+05,2016-09-25
4,0,8.354286e+05,2016-10-25
...,...,...,...
60146,2136,2.012214e+06,2018-10-25
60147,2136,2.135429e+06,2018-11-25
60148,2136,2.427429e+06,2018-12-25
60149,2136,1.873643e+06,2019-01-25


In [12]:
series  = finalset.loc[finalset['store_id'] == 0]

In [23]:
# 매출 데이터량(12개 미만)이 적은 가게를 필터링하는 함수 / 제거 알고리즘 구현해야함
n_store = 0 
for i in finalset['store_id'].unique()[30:]:
    subset_5 = finalset[finalset['store_id'] == i]
    if len(subset_5) < 12:
        print(i,"매출 데이터량이 너무 적습니다.")
        n_store += 1
    else:
        pass
print("매출 데이터량이 적은 가게의 수 : ", n_store)

795 매출 데이터량이 너무 적습니다.
매출 데이터량이 적은 가게의 수 :  1


In [269]:
# 2019-02 월 데이터가 없는 가게를 필터링하는 함수 / 제거 알고리즘 구현해야함

n_store = 0
for i in finalset['store_id'].unique()[30:]:
    subset_5 = finalset[finalset['store_id'] == i].tail(1)
    if finalset['datetime'][32] != subset_5['datetime'].values:
        print(i,"번째 가게는 최근 30일 간의 매출이 없습니다.")
        print(subset_5)
        n_store += 1
    else:
        pass
print(n_store,"의 가게가 최근 매출이 없습니다.")

34 번째 가게는 최근 30일 간의 매출이 없습니다.
      store_id         amount             datetime
1109        34  260957.142857  2019-01-25 00:00:00
111 번째 가게는 최근 30일 간의 매출이 없습니다.
      store_id   amount             datetime
3376       111  83000.0  2018-09-25 00:00:00
251 번째 가게는 최근 30일 간의 매출이 없습니다.
      store_id        amount             datetime
7529       251  42857.142857  2019-01-25 00:00:00
257 번째 가게는 최근 30일 간의 매출이 없습니다.
      store_id         amount             datetime
7711       257  276785.714286  2018-09-25 00:00:00
277 번째 가게는 최근 30일 간의 매출이 없습니다.
      store_id        amount             datetime
8288       277  23142.857143  2019-01-25 00:00:00
368 번째 가게는 최근 30일 간의 매출이 없습니다.
       store_id        amount             datetime
11016       368  81471.428571  2019-01-25 00:00:00
374 번째 가게는 최근 30일 간의 매출이 없습니다.
       store_id         amount             datetime
11211       374  169571.428571  2018-12-25 00:00:00
479 번째 가게는 최근 30일 간의 매출이 없습니다.
       store_id   amount             datetime
14140  

In [25]:
# datetime type 을 datetime[ns]로 변경해야함

for i in tqdm(finalset['store_id'].unique()[741:744]): 
    try:
        series  = finalset.loc[finalset['store_id'] == i]
        model = auto_arima(np.array(series['amount']), start_p=0, start_q=0,supress_warnings=False)
        # model_fit = model.fit(trend='c',full_output=True, disp=1)
        forecast_data = model.predict(n_periods=3)

        subset_4 = pd.DataFrame(data={'store_id': [],'amount' : [],'datetime' : []})
        subset_4 = subset_4.append({'amount' : forecast_data.tolist()[0], 'datetime' : '2019-03-25'},  ignore_index=True)
        subset_4 = subset_4.append({'amount' : forecast_data.tolist()[1], 'datetime' : '2019-04-25'},  ignore_index=True)
        subset_4 = subset_4.append({'amount' : forecast_data.tolist()[2], 'datetime' : '2019-05-25'},  ignore_index=True)
        subset_4['store_id'] = i
        finalset = pd.concat([finalset, subset_4], axis = 0)
        finalset = finalset.sort_values(by=['store_id','datetime']).reset_index(drop = True)
    except:
        print(i,'번째에서 에러')
finalset

 33%|███▎      | 1/3 [00:00<00:01,  1.72it/s]

795 번째에서 에러


100%|██████████| 3/3 [00:00<00:00,  3.08it/s]


,store_id,amount,datetime
0,0,7.470000e+05,2016-06-25
1,0,1.005000e+06,2016-07-25
2,0,8.715714e+05,2016-08-25
3,0,8.978571e+05,2016-09-25
4,0,8.354286e+05,2016-10-25
...,...,...,...
62384,2136,2.012214e+06,2018-10-25
62385,2136,2.135429e+06,2018-11-25
62386,2136,2.427429e+06,2018-12-25
62387,2136,1.873643e+06,2019-01-25


In [115]:
series['amount'] = forecast_data[0]
series['year_month'] = pd.to_datetime('2019-03-25', format='%Y-%m-%d')
series['store_id'] = 0

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (Temp/ipykernel_24544/4230796615.py, line 2)